In [9]:
# Install dependencies
!pip install tensorflow gradio numpy matplotlib scikit-learn

# Imports
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
import gradio as gr
import matplotlib.pyplot as plt

# Generate synthetic sine wave data
time = np.linspace(0, 100, 1000)
data = np.sin(time)  # Sine wave data

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data.reshape(-1, 1))

# Create the dataset for training
def create_dataset(data, time_step=50):
    X, y = [], []
    for i in range(len(data)-time_step-1):
        X.append(data[i:(i+time_step), 0])
        y.append(data[i+time_step, 0])
    return np.array(X), np.array(y)

# Prepare data for LSTM
time_step = 50
X, y = create_dataset(data_scaled, time_step)
X = X.reshape(X.shape[0], X.shape[1], 1)  # Reshape for LSTM input

# Split data into train and test sets
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Build LSTM model
def build_lstm_model():
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=False, input_shape=(X_train.shape[1], 1)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Initialize and train the model
model = build_lstm_model()
model.fit(X_train, y_train, epochs=10, batch_size=32)

# Prediction function
def predict_next_value(input_sequence):
    # Rescale input sequence and reshape for prediction
    input_sequence_scaled = scaler.transform(np.array(input_sequence).reshape(-1, 1))
    input_sequence_scaled = input_sequence_scaled.reshape(1, time_step, 1)

    # Get prediction
    prediction_scaled = model.predict(input_sequence_scaled)

    # Inverse scale to get actual value
    prediction = scaler.inverse_transform(prediction_scaled)

    return f"Predicted next value: {prediction[0][0]:.4f}"

# Gradio interface
gr.Interface(
    fn=predict_next_value,
    inputs=gr.Slider(minimum=-1.0, maximum=1.0, step=0.01, label="Input Sequence for Prediction", value=0),
    outputs="text",
    title="📊 Time Series Prediction with LSTM",
    description="Predict the next value in a sine wave time series using LSTM"
).launch()


Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - loss: 0.1909
Epoch 2/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 0.0271
Epoch 3/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 0.0023
Epoch 4/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 3.4902e-04
Epoch 5/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 9.3640e-05
Epoch 6/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 5.6663e-05
Epoch 7/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 4.9331e-05
Epoch 8/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 4.5456e-05
Epoch 9/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - loss: 4.3996e-05
Epoch 10/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 4.1662e-05
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* R